In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time

player = {'Season' : 0, 'Name' : '', 'Position' : '', 'Age' : 0, 'Fee (€ 1Th)' : 0}
player_df = pd.DataFrame(columns=['Season', 'Name', 'Position', 'Age', 'Fee (€ 1Th)'])
stats = {'Season' : 0, 'Name' : '', 'Position' : '', 'Age' : 0, 
         'Competition' : '', 'Appearances' : 0, 'Goals' : 0, 'Assists' : 0, 'Yellow_cards' : 0,
         'Second_yellow_cards' : 0, 'Red_cards' : 0, 'Penalty_goals' : 0, 
         'Minutes_per_goal' : 0, 'Minutes_played' : 0}
stats_df = pd.DataFrame(columns=stats.keys())
stats_df

In [ ]:
for page in range(1, 21):
    url = f'https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop?ajax=yw1&page={page}'
    header = {
        'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'
    }
    
    seed = np.random.randint(100) 
    np.random.seed(seed) 
    a = np.random.randint(2) 
    time.sleep(a) 

    resp = requests.get(url, headers = header)
    html = BeautifulSoup(resp.text, 'html.parser')

    for item in html.find_all('tr', class_ = ['odd', 'even']):
        player['Name'] = item.find_all('td')[3].text
        player['Position'] =item.find_all('td')[4].text
        age_22 =int(item.find_all('td')[5].text)
    
        item_url = 'https://www.transfermarkt.com' + item.find_all('td')[3].find('a').attrs['href']
        detail_referer = item_url.replace('profil', 'detaillierteleistungsdaten')
        detail_url = detail_referer + '/plus/1'
               
                    
        seed = np.random.randint(100) 
        np.random.seed(seed) 
        a = np.random.randint(2) 
        time.sleep(a)
        
        header = {
            'referer' : 'https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop',
            'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'   
        }
        
        item_resp = requests.get(item_url, headers = header)
        
        item_html = BeautifulSoup(item_resp.text, 'html.parser')
        
        for season_item, fee_item in zip(item_html.find_all('div', class_='tm-player-transfer-history-grid__season')[1 : -1], item_html.find_all('div', class_='tm-player-transfer-history-grid__fee')[1 : -1]):            
            fee_col = fee_item.text.strip()
            season_col = season_item.text.strip()
            
            if '/' in season_col:
                season = int('20' + season_col.split('/')[0])
            else:
                season = int(season_col)
                
            player['Season'] = season
            player['Age'] = age_22 - (2022 - season)
            
            try:
                if fee_col[0] == '€' and fee_col.find('m') != -1:
                    fee_ = fee_col.split('.')
                    fee = int(fee_[0][1 : ] + fee_[1][ : -1]) * 10
                    player['Fee (€ 1Th)'] = fee
                elif fee_col[0] == '€' and fee_col.find('Th'):
                    fee_ = fee_col.split('Th')
                    fee = int(fee_[0][1 : ])
                    player['Fee (€ 1Th)'] = fee
                else:
                    continue
#                 print(player)
                player_df = player_df.append(player, ignore_index=True)
            except:
                pass
            
        header_detail = {
            'header' : detail_referer,
            'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'
        }
        
        seed = np.random.randint(100) 
        np.random.seed(seed) 
        a = np.random.randint(2) 
        time.sleep(a)
        
        detail_resp = requests.get(detail_url, headers = header)
        detail_html = BeautifulSoup(detail_resp.text, 'html.parser')
        
        stats['Name'] = player['Name']
        stats['Position'] = player['Position']
        
        for box in detail_html.find_all('div', class_='box')[1 : ]:
            datas = box.find_all('tr', class_=['odd', 'even'])
            
            for data in datas:
                for index, col in enumerate(data.find_all('td')):
                    if index == 0:
                        if '/' in col.text.strip():
                            stats['Season'] = int('20' + col.text.strip().split('/')[0])
                        else:
                            stats['Season'] = int(col.text.strip())
                        stats['Age'] = age_22 - (2022 - stats['Season'])
                    elif index == 2:
                        stats['Competition'] = col.find('a').text.strip()
                    elif index == 4:
                        try:
                            stats['Appearances'] = int(col.find('a').text.strip())
                        except:
                            stats['Appearances'] = 0
                    elif index == 5:
                        try:
                            stats['Goals'] = int(col.text.strip())
                        except:
                            stats['Goals'] = 0
                    elif index == 6:
                        try:
                            stats['Assists'] = int(col.text.strip())
                        except:
                            stats['Assists'] = 0

                    elif index == 10:
                        try:
                            stats['Yellow_cards'] = int(col.text.strip())
                        except:
                            stats['Yellow_cards'] = 0
                    elif index == 11:
                        try:
                            stats['Second_yellow_cards'] = int(col.text.strip())
                        except:
                            stats['Second_yellow_cards'] = 0
                    elif index == 12:
                        try:
                            stats['Red_cards'] = int(col.text.strip())
                        except:
                            stats['Red_cards'] = 0
                    elif index == 13:
                        try:
                            stats['Penalty_goals'] = int(col.text.strip())
                        except:
                            stats['Penalty_goals'] = 0
                    elif index == 14:
                        if "'" in col.text:
                            if (".") in col.text:
                                stats['Minutes_per_goal'] = int(col.text.strip().split('.')[0])
                            else:
                                stats['Minutes_per_goal'] = int(col.text.strip().split("'")[0])
                        else:
                            stats['Minutes_per_goal'] = 0
                    elif index == 15:
                        if "'" in col.text:
                            if "." in col.text:
                                stats['Minutes_played'] = int(col.text.strip().split('.')[0])
                            else:
                                stats['Minutes_played'] = int(col.text.strip().split("'")[0])
                        else:
                            stats['Minutes_played'] = 0
                stats_df = stats_df.append(stats, ignore_index=True)
#                 print(stats)

In [ ]:
player_df.to_excel('player_fee.xlsx')